In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel
import warnings

# Suppress numerical warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# -------------------------------------------------
# ✅ Raw Data (5 Random Seeds for each model)
# -------------------------------------------------
data = {
    "ShuffleNet V2": {
        "CO2 (kg)": [0.009835, 0.009665, 0.009702, 0.009722, 0.009772],
        "Energy (kWh)": [0.028161, 0.027674, 0.027780, 0.027839, 0.027983]
    },
    "MobileNet V3": {
        "CO2 (kg)": [0.009586, 0.009746, 0.009805, 0.009870, 0.009915],
        "Energy (kWh)": [0.020362, 0.020701, 0.020826, 0.020965, 0.021061]
    },
    "EfficientNet V2-S": {
        "CO2 (kg)": [0.007559, 0.007639, 0.007628, 0.007637, 0.007643],
        "Energy (kWh)": [0.021644, 0.021875, 0.021842, 0.021869, 0.021886]
    },
    "Ensemble MESNET": {
        "CO2 (kg)": [0.006863, 0.006835, 0.006799, 0.006905, 0.007014],
        "Energy (kWh)": [0.014579, 0.014518, 0.014441, 0.014666, 0.014898]
    }
}

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel
import warnings

# Suppress numerical warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# -------------------------------------------------
# ✅ Raw Data (5 Random Seeds for each model)
# -------------------------------------------------
data = {
    "ShuffleNet V2": {
        "CO2 (kg)": [0.009835, 0.009665, 0.009702, 0.009722, 0.009772],
        "Energy (kWh)": [0.028161, 0.027674, 0.027780, 0.027839, 0.027983]
    },
    "MobileNet V3": {
        "CO2 (kg)": [0.009586, 0.009746, 0.009805, 0.009870, 0.009915],
        "Energy (kWh)": [0.020362, 0.020701, 0.020826, 0.020965, 0.021061]
    },
    "EfficientNet V2-S": {
        "CO2 (kg)": [0.007559, 0.007639, 0.007628, 0.007637, 0.007643],
        "Energy (kWh)": [0.021644, 0.021875, 0.021842, 0.021869, 0.021886]
    },
    "Ensemble MESNET": {
        "CO2 (kg)": [0.006863, 0.006835, 0.006799, 0.006905, 0.007014],
        "Energy (kWh)": [0.014579, 0.014518, 0.014441, 0.014666, 0.014898]
    }
}

# -------------------------------------------------
# ✅ Compute Mean ± SD
# -------------------------------------------------
rows = []
for model, metrics in data.items():
    co2_mean = np.mean(metrics["CO2 (kg)"])
    co2_std = np.std(metrics["CO2 (kg)"], ddof=1)
    energy_mean = np.mean(metrics["Energy (kWh)"])
    energy_std = np.std(metrics["Energy (kWh)"], ddof=1)
    
    rows.append([
        model,
        f"{co2_mean:.6f} ± {co2_std:.6f}",
        f"{energy_mean:.6f} ± {energy_std:.6f}"
    ])

# Convert to DataFrame
df = pd.DataFrame(rows, columns=["Model", "CO2 (kg)", "Energy (kWh)"])

# -------------------------------------------------
# ✅ Display Table (Word/Excel Ready)
# -------------------------------------------------
print("\n=== Mean ± SD for Each Model ===\n")
print(df.to_string(index=False))



=== Mean ± SD for Each Model ===

            Model            CO2 (kg)        Energy (kWh)
    ShuffleNet V2 0.009739 ± 0.000066 0.027887 ± 0.000189
     MobileNet V3 0.009784 ± 0.000128 0.020783 ± 0.000272
EfficientNet V2-S 0.007621 ± 0.000035 0.021823 ± 0.000101
  Ensemble MESNET 0.006883 ± 0.000083 0.014620 ± 0.000176


In [2]:
# -------------------------------------------------
# ✅ Step 2: Safe p-value formatting
# -------------------------------------------------
def format_pvalue(p):
    if np.isnan(p):
        return "NaN"
    elif p == 0.0 or p < 1e-308:
        return "< 1 × 10⁻³⁰⁸"
    elif p < 0.0001:
        exponent = int(np.floor(np.log10(p)))
        base = p / (10 ** exponent)
        return f"{base:.2f} × 10⁻{abs(exponent)}"
    else:
        return f"{p:.4f}"

In [3]:
# -------------------------------------------------
# ✅ Step 3: Paired t-test vs Ensemble MESNET (only CO₂ & Energy)
# -------------------------------------------------
ensemble = data["Ensemble MESNET"]
results = []

for model in ["ShuffleNet V2", "MobileNet V3", "EfficientNet V2-S"]:
    row = {"Comparison": model}
    for metric in ["CO2 (kg)", "Energy (kWh)"]:
        t_stat, p_val = ttest_rel(data[model][metric], ensemble[metric])
        significance = "✔️ Significant (p<0.05)" if p_val < 0.05 else "❌ Not Significant"
        row[f"{metric} p-value"] = format_pvalue(p_val)
        row[f"{metric} Significance"] = significance
    results.append(row)

df_ttest = pd.DataFrame(results)

print("\n\n=== Paired t-test Results vs Ensemble MESNET ===\n")
print(df_ttest.to_string(index=False))



=== Paired t-test Results vs Ensemble MESNET ===

       Comparison CO2 (kg) p-value   CO2 (kg) Significance Energy (kWh) p-value Energy (kWh) Significance
    ShuffleNet V2      1.70 × 10⁻7 ✔️ Significant (p<0.05)          1.22 × 10⁻8   ✔️ Significant (p<0.05)
     MobileNet V3      4.65 × 10⁻7 ✔️ Significant (p<0.05)          4.69 × 10⁻7   ✔️ Significant (p<0.05)
EfficientNet V2-S      3.45 × 10⁻5 ✔️ Significant (p<0.05)          9.15 × 10⁻8   ✔️ Significant (p<0.05)
